In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings
import numpy as np
from PIL import Image
warnings.filterwarnings('ignore')
%matplotlib inline

In [3]:
data_dr = 'Polyp_dataset'
meta_data = pd.read_csv(os.path.join(data_dr,'metadata.csv'))
meta_data

,frame_id,sequence_id,tif_image_path,tif_mask_path,png_image_path,png_mask_path
0,1,1,TIFF/Original/1.tif,TIFF/Ground Truth/1.tif,PNG/Original/1.png,PNG/Ground Truth/1.png
1,10,1,TIFF/Original/10.tif,TIFF/Ground Truth/10.tif,PNG/Original/10.png,PNG/Ground Truth/10.png
2,100,5,TIFF/Original/100.tif,TIFF/Ground Truth/100.tif,PNG/Original/100.png,PNG/Ground Truth/100.png
3,101,5,TIFF/Original/101.tif,TIFF/Ground Truth/101.tif,PNG/Original/101.png,PNG/Ground Truth/101.png
4,102,5,TIFF/Original/102.tif,TIFF/Ground Truth/102.tif,PNG/Original/102.png,PNG/Ground Truth/102.png
...,...,...,...,...,...,...
607,95,5,TIFF/Original/95.tif,TIFF/Ground Truth/95.tif,PNG/Original/95.png,PNG/Ground Truth/95.png
608,96,5,TIFF/Original/96.tif,TIFF/Ground Truth/96.tif,PNG/Original/96.png,PNG/Ground Truth/96.png
609,97,5,TIFF/Original/97.tif,TIFF/Ground Truth/97.tif,PNG/Original/97.png,PNG/Ground Truth/97.png
610,98,5,TIFF/Original/98.tif,TIFF/Ground Truth/98.tif,PNG/Original/98.png,PNG/Ground Truth/98.png


In [4]:
class_data = pd.read_csv(os.path.join(data_dr,'class_dict.csv'))
class_data

,class_names,r,g,b
0,background,0,0,0
1,polyp,255,255,255


In [5]:
metadata_df = meta_data[['frame_id','png_image_path','png_mask_path']]
metadata_df['png_image_path'] = metadata_df['png_image_path'].apply(lambda img_path: os.path.join(data_dr, img_path))
metadata_df['png_mask_path'] = metadata_df['png_mask_path'].apply(lambda img_path: os.path.join(data_dr, img_path))
metadata_df = metadata_df.sample(frac = 1).reset_index(drop = True)
metadata_df

,frame_id,png_image_path,png_mask_path
0,24,Polyp_dataset/PNG/Original/24.png,Polyp_dataset/PNG/Ground Truth/24.png
1,382,Polyp_dataset/PNG/Original/382.png,Polyp_dataset/PNG/Ground Truth/382.png
2,290,Polyp_dataset/PNG/Original/290.png,Polyp_dataset/PNG/Ground Truth/290.png
3,213,Polyp_dataset/PNG/Original/213.png,Polyp_dataset/PNG/Ground Truth/213.png
4,308,Polyp_dataset/PNG/Original/308.png,Polyp_dataset/PNG/Ground Truth/308.png
...,...,...,...
607,298,Polyp_dataset/PNG/Original/298.png,Polyp_dataset/PNG/Ground Truth/298.png
608,215,Polyp_dataset/PNG/Original/215.png,Polyp_dataset/PNG/Ground Truth/215.png
609,429,Polyp_dataset/PNG/Original/429.png,Polyp_dataset/PNG/Ground Truth/429.png
610,241,Polyp_dataset/PNG/Original/241.png,Polyp_dataset/PNG/Ground Truth/241.png


In [6]:
img_paths = metadata_df['png_image_path'].tolist()
mask_paths = metadata_df['png_mask_path'].tolist()


In [7]:
# Define a function to load and preprocess an image
def load_and_preprocess_image(image_path, target_size):
    img = Image.open(image_path)
    img = img.resize(target_size)  # Resize the image to the desired size
    img_array = np.array(img) / 255.0  # Normalize pixel values to the range [0, 1]
    return img_array

# Specify the target size for resizing (optional)
target_size = (256,256)

# Load and preprocess each image and mask
images = [load_and_preprocess_image(image_path, target_size=target_size) for image_path in img_paths]
masks = [load_and_preprocess_image(image_path, target_size=target_size) for image_path in mask_paths]

# Convert the list of images to a NumPy array
images_array = np.array(images)
mask_array = np.array(masks)

print("Shape of the images array:", images_array.shape)
print("Shape of the images array:", mask_array.shape)


Shape of the images array: (612, 256, 256, 3)
Shape of the images array: (612, 256, 256, 3)


In [8]:
x_train_temp, x_test, y_train_temp, y_test = train_test_split(images_array, mask_array, test_size=0.1, random_state=42)

x_train, x_valid, y_train, y_valid = train_test_split(x_train_temp, y_train_temp, test_size=0.15, random_state=42)

# Print the sizes of the sets to verify the split
print("Training set size:", len(x_train))
print("Validation set size:", len(x_valid))
print("Test set size:", len(x_test))


Training set size: 467
Validation set size: 83
Test set size: 62


In [9]:
# Create TensorFlow datasets
#train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))

#test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

In [10]:
def visualize(images):

    n_images = len(images)
    plt.figure(figsize=(20,8))
    for idx, (name, image) in enumerate(images.items()):
        plt.subplot(1, n_images, idx + 1)
        plt.xticks([]);
        plt.yticks([])
        # get title from the parameter names
        plt.title(name.replace('_',' ').title(), fontsize=20)
        plt.imshow(image)
    plt.show()

In [11]:
def munet_model(input_shape):
    inputs = layers.Input(input_shape)

    mconv1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(mconv1)

    mconv2 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(pool1)
    mconv2 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(mconv2)

    up1 = layers.UpSampling2D(size=(2, 2))(mconv2)
    up1 = layers.concatenate([mconv1, up1], axis=-1)
    mconv3 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(up1)


    # Encoder
    conv1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(mconv3)
    conv1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
    pool4 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)

    # Bottleneck
    conv4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(conv4)

    # Decoder
    up5 = layers.UpSampling2D(size=(2, 2))(conv4)
    up5 = layers.concatenate([conv3, up5], axis=-1)
    conv5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(up5)
    conv5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(conv5)

    up6 = layers.UpSampling2D(size=(2, 2))(conv5)
    up6 = layers.concatenate([conv2, up6], axis=-1)
    conv6 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(up6)
    conv6 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv6)

    up7 = layers.UpSampling2D(size=(2, 2))(conv6)
    up7 = layers.concatenate([conv1, up7], axis=-1)
    conv7 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(up7)
    conv7 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv7)

    nconv1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv7)
    pool5 = layers.MaxPooling2D(pool_size=(2, 2))(nconv1)

    nconv2 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(pool5)
    nconv2 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(nconv2)

    up2 = layers.UpSampling2D(size=(2, 2))(nconv2)
    up2 = layers.concatenate([nconv1, up2], axis=-1)
    nconv3 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(up2)

    output = layers.Conv2D(3, (1, 1), activation='sigmoid')(nconv3)

    model = models.Model(inputs=inputs, outputs=output)

    return model

# Create an instance of the U-Net model
model = munet_model(input_shape=(256, 256, 3))

# Display the model summary
model.summary()

2024-02-28 15:15:07.502047: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-02-28 15:15:07.502093: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-02-28 15:15:07.503722: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-02-28 15:15:07.504209: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-28 15:15:07.505063: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 256, 256, 64)         1792      ['input_1[0][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 128, 128, 64)         0         ['conv2d[0][0]']              
 D)                                                                                               
                                                                                                  
 conv2d_1 (Conv2D)           (None, 128, 128, 256)        147712    ['max_pooling2d[0][0]']   

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(x_train, y_train,batch_size = 10, epochs= 150 ,shuffle = True, validation_data=(x_valid,y_valid))

Epoch 1/150


2024-02-28 15:15:11.145993: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


30/47 [==================>...........] - ETA: 1:11 - loss: 1.1813 - accuracy: 0.4516

In [ ]:
loss, accuracy =model.evaluate(x_test,y_test)

In [ ]:

y_pred =model.evaluate(x_test)
y_pred

In [ ]:

import matplotlib.pyplot as plot
plot.subplot(2,1,2)
plot.plot(history.history['loss'])
plot.plot(history.history['val_loss'])
plot.title('model loss')
plot.ylabel('loss')
plot.xlabel('epoch')
plot.legend(['train', 'val'], loc='upper right')

In [ ]:
import matplotlib.pyplot as plot
plot.subplot(2,1,1)
plot.plot(history.history['accuracy'])
plot.plot(history.history['val_accuracy'])
plot.title('model accuracy')
plot.ylabel('accuracy')
plot.xlabel('epoch')
plot.legend(['train', 'val'], loc='lower right')

In [ ]:
# Predict the mask using the trained model
y_pred = model.predict(x_test)

# Load the original mask
original_mask = y_test
original_mask = np.expand_dims(original_mask, axis=0) / 255.0  # Normalize the mask

# Plot the original and predicted masks side by side
plt.figure(figsize=(12, 6))

# Original Mask
plt.subplot(1, 2, 1)
plt.imshow(original_mask[0, :, :, 0], cmap='gray')
plt.title('Original Mask')

# Predicted Mask
plt.subplot(1, 2, 2)
plt.imshow(y_pred[0, :, :, 0], cmap='gray')
plt.title('Predicted Mask')

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from tensorflow.keras.preprocessing import image



# Threshold predicted masks (assuming binary segmentation)
y_pred_binary = (y_pred > 0.2)

y_test_flat = y_test.flatten().astype(np.uint8)
y_pred_flat = y_pred_binary.flatten().astype(np.uint8)

# Calculate metrics
accuracy = accuracy_score(y_test_flat, y_pred_flat)
precision = precision_score(y_test_flat, y_pred_flat)
recall = recall_score(y_test_flat, y_pred_flat)

# Intersection over Union (IoU) or Jaccard Index
intersection = np.sum(y_test_flat * y_pred_flat)
union = np.sum((y_test_flat + y_pred_flat) > 0)
iou = intersection / union

# Confusion matrix
conf_mat = confusion_matrix(y_test_flat, y_pred_flat)

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'IoU: {iou:.4f}')
print(f'Confusion Matrix:\n{conf_mat}')

# Visualize model predictions on test images
num_samples = min(x_test.shape[0], 5)  # Visualize predictions for a few samples

for i in range(num_samples):
    plt.figure(figsize=(12, 4))

    # Original Image
    plt.subplot(1, 3, 1)
    plt.imshow(x_test[i])
    plt.title('Original Image')

    # Ground Truth Mask
    plt.subplot(1, 3, 2)
    plt.imshow(y_test[i, :, :, 0], cmap='gray')
    plt.title('Ground Truth Mask')

    # Predicted Mask
    plt.subplot(1, 3, 3)
    plt.imshow(y_pred_binary[i, :, :, 0])
    plt.title('Predicted Mask')

    plt.show()